In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import statistics
import math
import collections
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## Helper Functions

In [2]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img,lines


def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Draw Lines

In [3]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=5):
  
    left_line_info,right_line_info = averages_info(lines)
    right_l = get_the_line(get_lines(lines,
                              right_line_info['m_slope'],
                              check_variance(right_line_info['v_slope']),
                              right_line_info['x1s']))
    
    left_l = get_the_line(get_lines(lines,
                              left_line_info['m_slope'],
                              check_variance(left_line_info['v_slope']),
                              left_line_info['x1s']),
                         True)
    

    new_lines = [right_l,left_l]
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

     

def get_the_line(lines,left=False):
    line = [[0,0,0,0]]
    lines = sorted(lines)
    if len(lines) > 1:
        # first point and last one
        line = [[
            lines[0][0][0],
            lines[0][0][1],
            lines[-1][0][2],
            lines[-1][0][3]
                ]]
 
    if len(line) > 0 and len(line[0]) > 3:
        if left:
            line = get_extend_l_line(line,average_slope(lines))
        else:
            line = get_extend_r_line(line,average_slope(lines))
        
    
    return line



def get_extend_l_line(line,avg_slope):
    result_line = []
    for x1,y1,x2,y2 in line:
            if x2 != x1: 
                # extend the line to the bottom 
                y1 = 539 # image height
                x1 =  -int(((y2-y1)/ avg_slope) - x2)
                #  extend the line to top
                y2 = 340
                x2 = int(((y2-y1) / avg_slope) + x1)
                result_line = [[x1,y1,x2,y2]]
                
    return result_line

def get_extend_r_line(line,avg_slope):
    result_line = []
    for x1,y1,x2,y2 in line:
            if x2 != x1: 
                # extend the line to the bottom 
                x1 = x1
                y1 = y1
                y2 = 539 # image height
                x2 = int(((y2-y1) / avg_slope) + x1)
                #  extend the line to top
                y1 = 340
                x1 =  -int(((y2-y1)/ avg_slope) - x2)
                result_line = [[x1,y1,x2,y2]]

                
    return result_line

    
def get_lines(lines,ave_slope, ratio,x1_average):
    re_lines = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x2 != x1 and not is_x1_destance_far_from_x1_average(x1,x1_average):
                slope = ((y2-y1)/(x2-x1))
                if ave_slope-ratio <= slope <= ave_slope+ratio:
                    re_lines.append([[x1,y1,x2,y2]])
    
    return re_lines


def is_x1_destance_far_from_x1_average(x1,x1_average):
     return abs(x1 - x1_average) > 150
    
    

# minimize the error if variance is too big
def check_variance(variance):
    if variance > .5:
        return variance/2
    elif variance < .1:
        return .3
    else:
        return variance
    
def average_slope(lines):
    avg_slope = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x2 != x1:
                slope = ((y2-y1)/(x2-x1))
                avg_slope.append(slope)
                
    if len(avg_slope) > 0:
        return  statistics.mean(avg_slope)
    
    return 0
                 
    
# find the average slop and x1 of right and left
# asumation positve slop is right and negative slop is left.
def averages_info(lines):
    right = []
    left = []
    right_xs= []
    left_xs= []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if x2 != x1:
                slope = ((y2-y1)/(x2-x1))
                if(slope > 0.01):
                    right.append(slope)
                    right_xs.append(x1)
                elif(slope < -0.01):
                    left.append(slope)
                    left_xs.append(x1)
    
    m_right = .5
    v_right = 0.1
    m_right_xs= 525
    if len(right) > 1:
        m_right = statistics.mean(right)
        v_right = statistics.variance(right,m_right)    # to avoid recalculation the mean
        m_right_xs = statistics.mean(right_xs)

    v_left = 0.1
    m_left = .5
    m_left_xs = 425
    if len(left) > 1:
        m_left = statistics.mean(left)
        v_left = statistics.variance(left,m_left)
        m_left_xs = statistics.mean(left_xs)


    return ({'m_slope':m_left,
             'v_slope':v_left,
             'x1s':m_left_xs
            }, 
            {'m_slope':m_right,
             'v_slope':v_right,
             'x1s':m_right_xs
            }
           )

 ## Build a Lane Finding Pipeline
 

In [4]:

def process_image(image):
    img_gry = grayscale(image)
    kernel_size = 5
    blur_gray = gaussian_blur(img_gry, kernel_size)
    low_threshold = 50
    high_threshold = 180
    edges = canny(blur_gray,low_threshold,high_threshold)
    imshape = image.shape
    vertices = np.array([[(15,imshape[0]),
                          (imshape[1]*.45, imshape[0]*.6), 
                          (imshape[1]*.55,  imshape[0]*.6), 
                          (imshape[1]-15,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)
    rho = 2
    theta = np.pi / 180
    threshold = 15
    min_line_length = 40
    max_line_gap = 20
    line_image,lines = hough_lines(masked_edges,rho,theta,threshold,min_line_length,max_line_gap)
    wei_img = weighted_img(line_image,image)
    return wei_img




## Test

In [ ]:
white_output = 'test_videos_output/solidWhiteRight2.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}"> 
</video>
""".format(white_output))

In [ ]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4') #.subclip(0,5)
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [5]:
challenge_output = 'test_videos_output/challenge.mp4'
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:17<00:00, 13.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 8.14 s, sys: 2.38 s, total: 10.5 s
Wall time: 20.1 s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))